##Problem 1

Write a web scraping script which takes an input of any film actor and gives the output of filmography of that actor in descending order.

In [ ]:
#importing all the dependencies.
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [ ]:
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
movies = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value')
        for b in soup.select('td.posterColumn span[name=ir]')]

In [ ]:
# create a empty list for storing
# movie information
list1 = []

# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):
	
	# Separating movie into: 'location',
	# 'title', 'year'
	movie_string = movies[index].get_text()
	movie = (' '.join(movie_string.split()).replace('.', ''))
	movie_title = movie[len(str(index))+1:-7]
	year = re.search('\((.*?)\)', movie_string).group(1)
	Location = movie[:len(str(index))-(len(movie))]
	data = {"Location": Location,
			"Movie_title": movie_title,
			"rating": ratings[index],
			"Year": year,
			"Star_cast": crew[index],
			}
	list1.append(data)


In [ ]:
for movie in list1:
    print(movie['Location'], '-', movie['Movie_title'], '('+movie['Year'] +
          ') -', 'Starring:', movie['Star_cast'], movie['rating'])

1 - The Shawshank Redemption (1994) - Starring: Frank Darabont (dir.), Tim Robbins, Morgan Freeman 9.233954495698477
2 - The Godfather (1972) - Starring: Francis Ford Coppola (dir.), Marlon Brando, Al Pacino 9.1560474473662
3 - The Dark Knight (2008) - Starring: Christopher Nolan (dir.), Christian Bale, Heath Ledger 8.985754553547302
4 - The Godfather Part II (1974) - Starring: Francis Ford Coppola (dir.), Al Pacino, Robert De Niro 8.984208074470846
5 - 12 Angry Men (1957) - Starring: Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb 8.947842593083273
6 - Schindler's List (1993) - Starring: Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes 8.935321404277829
7 - The Lord of the Rings: The Return of the King (2003) - Starring: Peter Jackson (dir.), Elijah Wood, Viggo Mortensen 8.921398500436839
8 - Pulp Fiction (1994) - Starring: Quentin Tarantino (dir.), John Travolta, Uma Thurman 8.850621769842494
9 - The Lord of the Rings: The Fellowship of the Ring (2001) - Starring: Peter Jackson (dir

In [ ]:
#saving the list as dataframe
df = pd.DataFrame(list1)

In [ ]:
df.head()

,Location,Movie_title,rating,Year,Star_cast
0,1,The Shawshank Redemption,9.233954495698477,1994,"Frank Darabont (dir.), Tim Robbins, Morgan Fre..."
1,2,The Godfather,9.1560474473662,1972,"Francis Ford Coppola (dir.), Marlon Brando, Al..."
2,3,The Dark Knight,8.985754553547302,2008,"Christopher Nolan (dir.), Christian Bale, Heat..."
3,4,The Godfather Part II,8.984208074470846,1974,"Francis Ford Coppola (dir.), Al Pacino, Robert..."
4,5,12 Angry Men,8.947842593083273,1957,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb"


In [ ]:
input_actor= input('Enter the name of Star cast: ' )

Enter the name of Star cast: Leonardo DiCaprio


In [ ]:
df1=df[df['Star_cast'].str.contains(input_actor)]
df_dec=df1.sort_values('Year', ascending= False)

In [ ]:
df_dec.head()

,Location,Movie_title,rating,Year,Star_cast
133,134,The Wolf of Wall Street,8.171738530666474,2013,"Martin Scorsese (dir.), Leonardo DiCaprio, Jon..."
12,13,Inception,8.732946893137395,2010,"Christopher Nolan (dir.), Leonardo DiCaprio, J..."
147,148,Shutter Island,8.1431349321367,2010,"Martin Scorsese (dir.), Leonardo DiCaprio, Emi..."
39,40,The Departed,8.475095342627219,2006,"Martin Scorsese (dir.), Leonardo DiCaprio, Mat..."
176,177,Catch Me If You Can,8.086318510026318,2002,"Steven Spielberg (dir.), Leonardo DiCaprio, To..."


##Problem 2


Create an ML algorithm to classify the planets as Candidate/False positive/Confirmed etc based on the column “koi_disposition”.

In [ ]:
#importing all the dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

##Exploratory Data Analysis

In [ ]:
#reading the dataset
data2 = pd.read_csv("/content/drive/MyDrive/Web Scrapping and Machine Learning/kepler_data.csv", skiprows=52)
data2.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [ ]:
#checking the shape of the dataset
print("No of rows:", data2.shape[0])
print("No of columns:", data2.shape[1])

No of rows: 9564
No of columns: 49


In [ ]:
# Checking for missing values
data2.isnull().sum()

kepid                   0
kepoi_name              0
kepler_name          7204
koi_disposition         0
koi_pdisposition        0
koi_score            1510
koi_fpflag_nt           0
koi_fpflag_ss           0
koi_fpflag_co           0
koi_fpflag_ec           0
koi_period              0
koi_period_err1       454
koi_period_err2       454
koi_time0bk             0
koi_time0bk_err1      454
koi_time0bk_err2      454
koi_impact            363
koi_impact_err1       454
koi_impact_err2       454
koi_duration            0
koi_duration_err1     454
koi_duration_err2     454
koi_depth             363
koi_depth_err1        454
koi_depth_err2        454
koi_prad              363
koi_prad_err1         363
koi_prad_err2         363
koi_teq               363
koi_teq_err1         9564
koi_teq_err2         9564
koi_insol             321
koi_insol_err1        321
koi_insol_err2        321
koi_model_snr         363
koi_tce_plnt_num      346
koi_tce_delivname     346
koi_steff             363
koi_steff_er

In [ ]:
#droping the columns with more null values.
data2.drop(['koi_teq_err1', 'koi_teq_err2','kepler_name'], axis=1, inplace=True)

In [ ]:
#summary if the columns
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 46 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   kepid              9564 non-null   int64  
 1   kepoi_name         9564 non-null   object 
 2   koi_disposition    9564 non-null   object 
 3   koi_pdisposition   9564 non-null   object 
 4   koi_score          8054 non-null   float64
 5   koi_fpflag_nt      9564 non-null   int64  
 6   koi_fpflag_ss      9564 non-null   int64  
 7   koi_fpflag_co      9564 non-null   int64  
 8   koi_fpflag_ec      9564 non-null   int64  
 9   koi_period         9564 non-null   float64
 10  koi_period_err1    9110 non-null   float64
 11  koi_period_err2    9110 non-null   float64
 12  koi_time0bk        9564 non-null   float64
 13  koi_time0bk_err1   9110 non-null   float64
 14  koi_time0bk_err2   9110 non-null   float64
 15  koi_impact         9201 non-null   float64
 16  koi_impact_err1    9110 

In [ ]:
num_data2=data2.select_dtypes(exclude='object')
cat_data2=data2.select_dtypes(include='object')

In [ ]:
for i in num_data2.columns:
  if num_data2[i].isnull().sum()>0:
    num_data2[i]=num_data2[i].fillna(num_data2[i].median())

In [ ]:
num_data2.isnull().sum()

kepid                0
koi_score            0
koi_fpflag_nt        0
koi_fpflag_ss        0
koi_fpflag_co        0
koi_fpflag_ec        0
koi_period           0
koi_period_err1      0
koi_period_err2      0
koi_time0bk          0
koi_time0bk_err1     0
koi_time0bk_err2     0
koi_impact           0
koi_impact_err1      0
koi_impact_err2      0
koi_duration         0
koi_duration_err1    0
koi_duration_err2    0
koi_depth            0
koi_depth_err1       0
koi_depth_err2       0
koi_prad             0
koi_prad_err1        0
koi_prad_err2        0
koi_teq              0
koi_insol            0
koi_insol_err1       0
koi_insol_err2       0
koi_model_snr        0
koi_tce_plnt_num     0
koi_steff            0
koi_steff_err1       0
koi_steff_err2       0
koi_slogg            0
koi_slogg_err1       0
koi_slogg_err2       0
koi_srad             0
koi_srad_err1        0
koi_srad_err2        0
ra                   0
dec                  0
koi_kepmag           0
dtype: int64

In [ ]:
for i in cat_data2.columns:
  if cat_data2[i].isnull().sum()>0:
    cat_data2[i]=cat_data2[i].fillna(cat_data2[i].mode()) 

In [ ]:
cat_data2.isnull().sum()

kepoi_name             0
koi_disposition        0
koi_pdisposition       0
koi_tce_delivname    346
dtype: int64

In [ ]:
data2['koi_tce_delivname'].mode()

0    q1_q17_dr25_tce
dtype: object

In [ ]:
#checking the unique values of the catecorical columns.
cat_data2.nunique()

kepoi_name           9564
koi_disposition         3
koi_pdisposition        2
koi_tce_delivname       3
dtype: int64

In [ ]:
#dummy encoding for catecorical columns.
dummies=pd.get_dummies(cat_data2.drop(['koi_disposition','kepoi_name'],axis=1), drop_first=True)
dummies.head()

,koi_pdisposition_FALSE POSITIVE,koi_tce_delivname_q1_q17_dr24_tce,koi_tce_delivname_q1_q17_dr25_tce
0,0,0,1
1,0,0,1
2,0,0,1
3,1,0,1
4,0,0,1


In [ ]:
dataFinal=pd.concat([num_data2, dummies], axis=1).reset_index(drop=True)

##Spliting into train and test.

In [ ]:
Y = data2['koi_disposition']
X = dataFinal.copy()

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain, Ytest = train_test_split(X,Y, test_size=0.3, random_state = 0)
print(Xtrain.shape)
print(Xtest.shape)
print(Ytrain.shape)
print(Ytest.shape)

(6694, 45)
(2870, 45)
(6694,)
(2870,)


##Feature Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
rfc = RandomForestClassifier(random_state = 0, n_estimators = 100)
rfc.fit(Xtrain, Ytrain)
imp_feat = list(zip(X.columns, rfc.feature_importances_))
imp_feat

[('kepid', 0.006958944776568857),
 ('koi_score', 0.13943117331141194),
 ('koi_fpflag_nt', 0.04770957634111057),
 ('koi_fpflag_ss', 0.04625966740683037),
 ('koi_fpflag_co', 0.04146704059557305),
 ('koi_fpflag_ec', 0.010897865427776035),
 ('koi_period', 0.015249926666323925),
 ('koi_period_err1', 0.012585791690652519),
 ('koi_period_err2', 0.010952632315680678),
 ('koi_time0bk', 0.008416751813139492),
 ('koi_time0bk_err1', 0.017283559999291376),
 ('koi_time0bk_err2', 0.014911172029220805),
 ('koi_impact', 0.009325284630249984),
 ('koi_impact_err1', 0.006825255859080945),
 ('koi_impact_err2', 0.006452833310366147),
 ('koi_duration', 0.016502864353415518),
 ('koi_duration_err1', 0.02318270927025351),
 ('koi_duration_err2', 0.026828728543039672),
 ('koi_depth', 0.011400930715694094),
 ('koi_depth_err1', 0.010607502011117695),
 ('koi_depth_err2', 0.011134969380491362),
 ('koi_prad', 0.030525855966966428),
 ('koi_prad_err1', 0.017016938616678006),
 ('koi_prad_err2', 0.009937845354763728),
 ('

In [ ]:
sel = SelectFromModel(RandomForestClassifier(random_state = 0, n_estimators = 100), threshold = 0.01)
sel.fit(Xtrain, Ytrain)

SelectFromModel(estimator=RandomForestClassifier(random_state=0),
                threshold=0.01)

In [ ]:
selected_feat= Xtrain.columns[(sel.get_support())]
print(len(selected_feat))

25


In [ ]:
print("Selected Features:\n", list(selected_feat))

Selected Features:
 ['koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_insol_err1', 'koi_model_snr', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg_err2', 'koi_srad_err1', 'koi_pdisposition_FALSE POSITIVE']


In [ ]:
Xnew = X[list(selected_feat)]
print(Xnew.shape)

(9564, 25)


In [ ]:
Xtrain1,Xtest1,Ytrain1, Ytest1 = train_test_split(Xnew,Y, test_size=0.3, random_state = 0)
print(Xtrain1.shape)
print(Xtest1.shape)
print(Ytrain1.shape)
print(Ytest1.shape)

(6694, 25)
(2870, 25)
(6694,)
(2870,)


##Model Training

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 0, n_estimators = 500, max_leaf_nodes = 6)
rfc.fit(Xtrain, Ytrain)

RandomForestClassifier(max_leaf_nodes=6, n_estimators=500, random_state=0)

In [ ]:
Ypred = rfc.predict(Xtest)
from sklearn.metrics import classification_report
cr1 = classification_report(Ytest, Ypred, digits = 4)
print(cr1)

                precision    recall  f1-score   support

     CANDIDATE     0.8248    0.7493    0.7852       710
     CONFIRMED     0.7802    0.8291    0.8039       702
FALSE POSITIVE     0.9858    1.0000    0.9928      1458

      accuracy                         0.8962      2870
     macro avg     0.8636    0.8595    0.8607      2870
  weighted avg     0.8957    0.8962    0.8953      2870



Ada Boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc=AdaBoostClassifier(random_state = 0, n_estimators = 500, learning_rate=0.001)
abc.fit(Xtrain,Ytrain)

AdaBoostClassifier(learning_rate=0.001, n_estimators=500, random_state=0)

In [ ]:
Ypred = abc.predict(Xtest)
from sklearn.metrics import classification_report
cr2 = classification_report(Ytest, Ypred, digits = 4)
print(cr2)

                precision    recall  f1-score   support

     CANDIDATE     0.8773    0.6141    0.7225       710
     CONFIRMED     0.6989    0.9060    0.7891       702
FALSE POSITIVE     0.9966    1.0000    0.9983      1458

      accuracy                         0.8815      2870
     macro avg     0.8576    0.8400    0.8366      2870
  weighted avg     0.8943    0.8815    0.8789      2870



XGBoost

In [ ]:
from xgboost import XGBClassifier
xgbc = XGBClassifier(random_state = 0, n_estimators = 600, max_leaf_nodes = 5)
xgbc.fit(Xtrain,Ytrain)

XGBClassifier(max_leaf_nodes=5, n_estimators=600, objective='multi:softprob')

In [ ]:
Ypred = xgbc.predict(Xtest)
from sklearn.metrics import classification_report
cr3 = classification_report(Ytest, Ypred, digits = 4)
print(cr3)

                precision    recall  f1-score   support

     CANDIDATE     0.8507    0.8183    0.8342       710
     CONFIRMED     0.8218    0.8476    0.8345       702
FALSE POSITIVE     0.9966    1.0000    0.9983      1458

      accuracy                         0.9178      2870
     macro avg     0.8897    0.8886    0.8890      2870
  weighted avg     0.9177    0.9178    0.9176      2870



##Sub-questions for Problem 2


1. Why did you choose the particular algorithm?

  Ans: I have tried various bagging, boosting machine learning algorithms.Random Forest classifier gave me 89%, AdaBoost gave 88% accuracy and I achieved 92% accuracy using XGBoost which is the highest accuracy so, XGBoost is the best model,This is a boosting algorithm which reduces the overfitting.

2. What are the different tuning methods used for the algorithm?

  Ans: I improved my model performance by increasing the number of trees and also tried with different values of max_leaf_nodes.

3. Did you consider any other choice of algorithm?Why or why not?

  Ans: I have tried various other Bagging and boosting machine learning classifiers such as:

     -Random Forest
     Random subsamples of data are selected for Random Forest where the growing happens in parallel and overfitting is reduced with the combination of several underfitting features in the algorithm.

     -Ada Boost
      One of the many advantages of the AdaBoost Algorithm is it is fast, simple and easy to program. Also, it has the flexibility to be combined with any machine learning algorithm
     
     -XGBoost
     Overfitting is reduced with the help of regularization parameters in XGBoost that helps to select features based on weak and strong features in the decision tree. Optimal values of each leaf are calculated and hence the overall gradient of the tree is given as the output.

4. What is the accuracy?

   Ans: I have achieved 92% accuracy using XGBoost Model.

5. What are the different types of metrics that can be used to evaluate the model?
   
   Ans: The different types of metrics that can be used to evaluate the models are:

  Accuracy: Accuracy is the number of correctly predicted data points out of all the data points.

  Precision: Precision refers to the number of true positives divided by the total number of positive predictions (i.e., the number of true positives plus the number of false positives).

  Recall: Mathematically, we define recall as the number of true positives divided by the number of true positives plus the number of false negatives.

  F1-score: F1 score is defined as the harmonic mean between precision and recall. F1 score is used when dealing with imbalanced data as accuracy is not the preferred metric in these cases.

**Thank You!**
